# v21.3 Training - Korean Neural Sparse Encoder

This notebook trains a SPLADE model with:
- **Filtered data**: IG/PMI/Cross-encoder filtered synonym pairs
- **Balanced hard negatives**: Easy/Medium/Hard difficulty
- **New evaluation metrics**: Recall@K, MRR, nDCG (no saturation)

## Changes from v21.2

| Feature | v21.2 | v21.3 |
|---------|-------|-------|
| Data Quality | ~50% noise | **< 10% noise** |
| Hard Negatives | Random | **Difficulty-balanced** |
| Evaluation | Binary (saturated 100%) | **Recall@K, MRR, nDCG** |
| Medical Data | Load failed | **All 4 configs loaded** |

In [ ]:
import sys
from pathlib import Path

def find_project_root():
    current = Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / "pyproject.toml").exists() or (parent / "src").exists():
            return parent
    return Path.cwd().parent.parent

PROJECT_ROOT = find_project_root()
sys.path.insert(0, str(PROJECT_ROOT))

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import json
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, get_scheduler
from tqdm.auto import tqdm
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt

# Set random seed
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

print(f"Project root: {PROJECT_ROOT}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Training configuration - v21.3
CONFIG = {
    # Model
    "model_name": "skt/A.X-Encoder-base",
    "max_length": 64,

    # Data - v21.3 filtered dataset
    "data_dir": PROJECT_ROOT / "dataset" / "v21.3_filtered_enhanced",

    # Training
    "batch_size": 64,
    "gradient_accumulation_steps": 2,
    "num_epochs": 25,
    "learning_rate": 3e-6,
    "warmup_ratio": 0.15,
    "max_grad_norm": 1.0,

    # Loss weights
    "lambda_self": 4.0,
    "lambda_synonym": 10.0,
    "lambda_margin": 2.5,
    "lambda_flops": 8e-3,
    "target_margin": 1.5,

    # FP16
    "use_fp16": True,

    # Output
    "output_dir": PROJECT_ROOT / "outputs" / "v21.3_korean_enhanced",
    
    # Evaluation
    "eval_k_values": [5, 10, 20],  # For Recall@K
    "eval_every_n_epochs": 1,
}

CONFIG["output_dir"].mkdir(parents=True, exist_ok=True)

print("Configuration (v21.3):")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## 1. Load Data

In [ ]:
from datasets import load_from_disk

# Load triplet dataset from 02_data_preparation.ipynb
dataset_path = CONFIG["data_dir"] / "triplet_dataset"
dataset = load_from_disk(str(dataset_path))

print(f"Dataset:")
print(dataset)

train_dataset_hf = dataset["train"]
val_dataset_hf = dataset["validation"]

print(f"\nTrain: {len(train_dataset_hf):,} triplets")
print(f"Validation: {len(val_dataset_hf):,} triplets")

# Sample
print("\nSample triplets:")
for i in range(min(5, len(train_dataset_hf))):
    item = train_dataset_hf[i]
    print(f"  {item['anchor']} -> {item['positive']} (neg: {item['negative']}, {item['difficulty']})")

In [ ]:
# Load filtered synonym pairs for evaluation
filtered_pairs_path = CONFIG["data_dir"] / "filtered_synonym_pairs.jsonl"
eval_synonym_pairs = []
with open(filtered_pairs_path, "r", encoding="utf-8") as f:
    for line in f:
        eval_synonym_pairs.append(json.loads(line.strip()))

print(f"Loaded {len(eval_synonym_pairs):,} synonym pairs for evaluation")

## 2. Define SPLADE Model

In [ ]:
class SPLADEModel(nn.Module):
    """SPLADE model for Korean sparse retrieval."""

    def __init__(self, model_name: str = "skt/A.X-Encoder-base"):
        super().__init__()
        self.model = AutoModelForMaskedLM.from_pretrained(model_name)
        self.config = self.model.config
        self.relu = nn.ReLU()

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """Forward pass."""
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # SPLADE activation: log(1 + ReLU(x))
        token_scores = torch.log1p(self.relu(logits))

        # Mask padding tokens
        mask = attention_mask.unsqueeze(-1).float()
        token_scores = token_scores * mask

        # Max pooling over sequence
        sparse_repr, _ = token_scores.max(dim=1)

        # Token weights for analysis
        token_weights = token_scores.max(dim=-1).values

        return sparse_repr, token_weights

## 3. Define Dataset and Collator

In [ ]:
class TripletDataset(Dataset):
    """Dataset for triplet-based training."""

    def __init__(self, hf_dataset, tokenizer, max_length: int = 64):
        self.hf_dataset = hf_dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self) -> int:
        return len(self.hf_dataset)

    def __getitem__(self, idx: int) -> Dict:
        item = self.hf_dataset[idx]
        return {
            "anchor": item["anchor"],
            "positive": item["positive"],
            "negative": item["negative"],
            "difficulty": item.get("difficulty", "unknown"),
        }


def triplet_collate_fn(batch: List[Dict], tokenizer, max_length: int = 64) -> Dict:
    """Collate function for triplet DataLoader."""
    anchors = [item["anchor"] for item in batch]
    positives = [item["positive"] for item in batch]
    negatives = [item["negative"] for item in batch]

    anchor_encodings = tokenizer(
        anchors, padding=True, truncation=True, max_length=max_length, return_tensors="pt"
    )
    positive_encodings = tokenizer(
        positives, padding=True, truncation=True, max_length=max_length, return_tensors="pt"
    )
    negative_encodings = tokenizer(
        negatives, padding=True, truncation=True, max_length=max_length, return_tensors="pt"
    )

    return {
        "anchor_input_ids": anchor_encodings["input_ids"],
        "anchor_attention_mask": anchor_encodings["attention_mask"],
        "positive_input_ids": positive_encodings["input_ids"],
        "positive_attention_mask": positive_encodings["attention_mask"],
        "negative_input_ids": negative_encodings["input_ids"],
        "negative_attention_mask": negative_encodings["attention_mask"],
    }

## 4. Define Loss Functions

In [ ]:
def compute_self_loss(
    sparse_repr: torch.Tensor,
    input_ids: torch.Tensor,
    attention_mask: torch.Tensor,
) -> torch.Tensor:
    """Self-reconstruction loss."""
    batch_size = input_ids.shape[0]
    loss = 0.0

    for i in range(batch_size):
        valid_mask = attention_mask[i].bool()
        token_ids = input_ids[i][valid_mask]
        token_ids = token_ids[1:-1]  # Remove CLS and SEP

        if len(token_ids) > 0:
            weights = sparse_repr[i][token_ids]
            loss -= weights.mean()

    return loss / batch_size


def compute_positive_loss(
    anchor_repr: torch.Tensor,
    positive_input_ids: torch.Tensor,
    positive_attention_mask: torch.Tensor,
) -> torch.Tensor:
    """Positive (synonym activation) loss."""
    batch_size = anchor_repr.shape[0]
    loss = 0.0

    for i in range(batch_size):
        valid_mask = positive_attention_mask[i].bool()
        token_ids = positive_input_ids[i][valid_mask]
        token_ids = token_ids[1:-1]

        if len(token_ids) > 0:
            positive_weights = anchor_repr[i][token_ids]
            loss -= positive_weights.mean()

    return loss / batch_size


def compute_triplet_margin_loss(
    anchor_repr: torch.Tensor,
    positive_repr: torch.Tensor,
    negative_repr: torch.Tensor,
    margin: float = 1.0,
) -> torch.Tensor:
    """Triplet margin loss."""
    pos_sim = F.cosine_similarity(anchor_repr, positive_repr)
    neg_sim = F.cosine_similarity(anchor_repr, negative_repr)
    loss = F.relu(margin - pos_sim + neg_sim)
    return loss.mean()


def compute_flops_loss(sparse_repr: torch.Tensor) -> torch.Tensor:
    """FLOPS regularization."""
    mean_activation = sparse_repr.mean(dim=0)
    return (mean_activation ** 2).sum()

## 5. Define Ranking Evaluation Metrics

Using Recall@K, MRR, and nDCG instead of saturating binary metrics.

In [ ]:
from collections import defaultdict

def evaluate_ranking_metrics(
    model: SPLADEModel,
    tokenizer,
    synonym_pairs: List[Dict],
    device: torch.device,
    k_values: List[int] = [5, 10, 20],
    sample_size: int = 500,
) -> Dict[str, float]:
    """
    Evaluate model with ranking metrics (Recall@K, MRR).
    
    For each anchor, rank all vocabulary terms by similarity and check
    if ground truth synonyms appear in top-K.
    """
    model.eval()
    
    # Build anchor to synonyms mapping
    anchor_to_synonyms = defaultdict(set)
    for pair in synonym_pairs:
        anchor_to_synonyms[pair["source"]].add(pair["target"])
    
    # Sample anchors for evaluation
    anchors = list(anchor_to_synonyms.keys())
    if len(anchors) > sample_size:
        anchors = random.sample(anchors, sample_size)
    
    # Get vocabulary
    vocab = list(tokenizer.get_vocab().keys())
    special_tokens = {tokenizer.pad_token, tokenizer.cls_token, 
                      tokenizer.sep_token, tokenizer.unk_token, tokenizer.mask_token}
    vocab = [t for t in vocab if t not in special_tokens and len(t) >= 2]
    
    # Sample vocabulary for efficiency
    vocab_sample = random.sample(vocab, min(5000, len(vocab)))
    
    # Compute vocabulary embeddings
    vocab_embeddings = []
    batch_size = 256
    
    with torch.no_grad():
        for i in range(0, len(vocab_sample), batch_size):
            batch = vocab_sample[i:i+batch_size]
            inputs = tokenizer(
                batch, padding=True, truncation=True, 
                max_length=32, return_tensors="pt"
            )
            inputs = {k: v.to(device) for k, v in inputs.items()}
            sparse_repr, _ = model(inputs["input_ids"], inputs["attention_mask"])
            vocab_embeddings.append(sparse_repr.cpu())
    
    vocab_embeddings = torch.cat(vocab_embeddings, dim=0)  # [vocab_size, hidden]
    
    # Evaluate each anchor
    recall_at_k = {k: [] for k in k_values}
    mrr_scores = []
    
    with torch.no_grad():
        for anchor in tqdm(anchors, desc="Evaluating", leave=False):
            # Get anchor embedding
            inputs = tokenizer(
                anchor, padding=True, truncation=True,
                max_length=32, return_tensors="pt"
            )
            inputs = {k: v.to(device) for k, v in inputs.items()}
            anchor_repr, _ = model(inputs["input_ids"], inputs["attention_mask"])
            anchor_repr = anchor_repr.cpu()
            
            # Compute similarities
            similarities = F.cosine_similarity(
                anchor_repr, vocab_embeddings, dim=1
            ).numpy()
            
            # Rank by similarity
            ranked_indices = np.argsort(-similarities)
            ranked_terms = [vocab_sample[idx] for idx in ranked_indices]
            
            # Ground truth synonyms
            true_synonyms = anchor_to_synonyms[anchor]
            
            # Recall@K
            for k in k_values:
                top_k = set(ranked_terms[:k])
                hits = len(top_k & true_synonyms)
                recall = hits / len(true_synonyms) if true_synonyms else 0
                recall_at_k[k].append(recall)
            
            # MRR: rank of first correct synonym
            first_rank = None
            for rank, term in enumerate(ranked_terms, 1):
                if term in true_synonyms:
                    first_rank = rank
                    break
            
            if first_rank is not None:
                mrr_scores.append(1.0 / first_rank)
            else:
                mrr_scores.append(0.0)
    
    model.train()
    
    # Compute averages
    results = {}
    for k in k_values:
        results[f"recall@{k}"] = np.mean(recall_at_k[k]) * 100
    results["mrr"] = np.mean(mrr_scores)
    
    return results

## 6. Initialize Model and Data

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(CONFIG["model_name"])
print(f"Tokenizer: {CONFIG['model_name']}")
print(f"Vocab size: {tokenizer.vocab_size:,}")

# Load model
model = SPLADEModel(CONFIG["model_name"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Model loaded on {device}")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Create datasets
train_dataset = TripletDataset(
    hf_dataset=train_dataset_hf,
    tokenizer=tokenizer,
    max_length=CONFIG["max_length"],
)

val_dataset = TripletDataset(
    hf_dataset=val_dataset_hf,
    tokenizer=tokenizer,
    max_length=CONFIG["max_length"],
)

# Create dataloaders
train_dataloader = DataLoader(
    train_dataset,
    batch_size=CONFIG["batch_size"],
    shuffle=True,
    collate_fn=lambda batch: triplet_collate_fn(batch, tokenizer, CONFIG["max_length"]),
    num_workers=0,
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=CONFIG["batch_size"],
    shuffle=False,
    collate_fn=lambda batch: triplet_collate_fn(batch, tokenizer, CONFIG["max_length"]),
    num_workers=0,
)

print(f"Train batches per epoch: {len(train_dataloader):,}")
print(f"Validation batches: {len(val_dataloader):,}")

## 7. Training Loop

In [ ]:
# Optimizer and scheduler
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=CONFIG["learning_rate"],
    weight_decay=0.01,
)

num_training_steps = len(train_dataloader) * CONFIG["num_epochs"] // CONFIG["gradient_accumulation_steps"]
num_warmup_steps = int(num_training_steps * CONFIG["warmup_ratio"])

scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps,
)

scaler = torch.amp.GradScaler("cuda", enabled=CONFIG["use_fp16"])

print(f"Training steps: {num_training_steps:,}")
print(f"Warmup steps: {num_warmup_steps:,}")

In [ ]:
# Training loop
history = []
best_mrr = 0
best_epoch = 0

print("Starting training...")
print("=" * 80)

for epoch in range(CONFIG["num_epochs"]):
    model.train()
    epoch_losses = {
        "total": 0.0,
        "self": 0.0,
        "positive": 0.0,
        "triplet_margin": 0.0,
        "flops": 0.0,
    }
    num_batches = 0
    
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{CONFIG['num_epochs']}")
    
    for step, batch in enumerate(progress_bar):
        anchor_input_ids = batch["anchor_input_ids"].to(device)
        anchor_attention_mask = batch["anchor_attention_mask"].to(device)
        positive_input_ids = batch["positive_input_ids"].to(device)
        positive_attention_mask = batch["positive_attention_mask"].to(device)
        negative_input_ids = batch["negative_input_ids"].to(device)
        negative_attention_mask = batch["negative_attention_mask"].to(device)
        
        with torch.amp.autocast("cuda", enabled=CONFIG["use_fp16"]):
            anchor_repr, _ = model(anchor_input_ids, anchor_attention_mask)
            positive_repr, _ = model(positive_input_ids, positive_attention_mask)
            negative_repr, _ = model(negative_input_ids, negative_attention_mask)
            
            loss_self = compute_self_loss(anchor_repr, anchor_input_ids, anchor_attention_mask)
            loss_positive = compute_positive_loss(anchor_repr, positive_input_ids, positive_attention_mask)
            loss_triplet_margin = compute_triplet_margin_loss(
                anchor_repr, positive_repr, negative_repr,
                margin=CONFIG["target_margin"]
            )
            loss_flops = compute_flops_loss(anchor_repr)
            
            total_loss = (
                CONFIG["lambda_self"] * loss_self +
                CONFIG["lambda_synonym"] * loss_positive +
                CONFIG["lambda_margin"] * loss_triplet_margin +
                CONFIG["lambda_flops"] * loss_flops
            )
            
            total_loss = total_loss / CONFIG["gradient_accumulation_steps"]
        
        scaler.scale(total_loss).backward()
        
        epoch_losses["total"] += total_loss.item() * CONFIG["gradient_accumulation_steps"]
        epoch_losses["self"] += loss_self.item()
        epoch_losses["positive"] += loss_positive.item()
        epoch_losses["triplet_margin"] += loss_triplet_margin.item()
        epoch_losses["flops"] += loss_flops.item()
        num_batches += 1
        
        if (step + 1) % CONFIG["gradient_accumulation_steps"] == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), CONFIG["max_grad_norm"])
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            optimizer.zero_grad()
        
        progress_bar.set_postfix({
            "loss": f"{total_loss.item() * CONFIG['gradient_accumulation_steps']:.4f}",
            "margin": f"{loss_triplet_margin.item():.4f}",
        })
    
    # Average losses
    for key in epoch_losses:
        epoch_losses[key] /= num_batches
    
    # Evaluate
    if (epoch + 1) % CONFIG["eval_every_n_epochs"] == 0:
        eval_results = evaluate_ranking_metrics(
            model, tokenizer, eval_synonym_pairs, device,
            k_values=CONFIG["eval_k_values"],
            sample_size=300,
        )
        
        print(f"\nEpoch {epoch + 1}/{CONFIG['num_epochs']}:")
        print(f"  Loss: total={epoch_losses['total']:.4f}, margin={epoch_losses['triplet_margin']:.4f}")
        print(f"  Eval: Recall@5={eval_results['recall@5']:.1f}%, "
              f"Recall@10={eval_results['recall@10']:.1f}%, MRR={eval_results['mrr']:.4f}")
        
        # Save history
        history.append({
            **epoch_losses,
            **eval_results,
        })
        
        # Save best model
        if eval_results["mrr"] > best_mrr:
            best_mrr = eval_results["mrr"]
            best_epoch = epoch + 1
            
            torch.save({
                "epoch": epoch + 1,
                "model_state_dict": model.state_dict(),
                "eval_results": eval_results,
                "config": CONFIG,
            }, CONFIG["output_dir"] / "best_model.pt")
            print(f"  -> New best model saved! (MRR={best_mrr:.4f})")

print("\n" + "=" * 80)
print(f"Training complete!")
print(f"Best model: Epoch {best_epoch} with MRR {best_mrr:.4f}")

In [ ]:
# Save final model
torch.save({
    "model_state_dict": model.state_dict(),
    "config": CONFIG,
}, CONFIG["output_dir"] / "final_model.pt")

# Save training history
with open(CONFIG["output_dir"] / "training_history.json", "w") as f:
    json.dump(history, f, indent=2, default=str)

print(f"Final model saved to: {CONFIG['output_dir']}")

## 8. Plot Training Curves

In [ ]:
if history:
    epochs = range(1, len(history) + 1)

    fig, axes = plt.subplots(2, 3, figsize=(15, 8))

    # Total loss
    axes[0, 0].plot(epochs, [h["total"] for h in history], "-o", color="#3498db")
    axes[0, 0].set_title("Total Loss")
    axes[0, 0].set_xlabel("Epoch")
    axes[0, 0].grid(True, alpha=0.3)

    # Triplet margin loss
    axes[0, 1].plot(epochs, [h["triplet_margin"] for h in history], "-o", color="#e74c3c")
    axes[0, 1].set_title("Triplet Margin Loss")
    axes[0, 1].set_xlabel("Epoch")
    axes[0, 1].grid(True, alpha=0.3)

    # FLOPS loss
    axes[0, 2].plot(epochs, [h["flops"] for h in history], "-o", color="#2ecc71")
    axes[0, 2].set_title("FLOPS Loss")
    axes[0, 2].set_xlabel("Epoch")
    axes[0, 2].grid(True, alpha=0.3)

    # Recall@5
    axes[1, 0].plot(epochs, [h["recall@5"] for h in history], "-o", color="#9b59b6")
    axes[1, 0].set_title("Recall@5 (%)")
    axes[1, 0].set_xlabel("Epoch")
    axes[1, 0].set_ylim(0, 100)
    axes[1, 0].grid(True, alpha=0.3)

    # Recall@10
    axes[1, 1].plot(epochs, [h["recall@10"] for h in history], "-o", color="#1abc9c")
    axes[1, 1].set_title("Recall@10 (%)")
    axes[1, 1].set_xlabel("Epoch")
    axes[1, 1].set_ylim(0, 100)
    axes[1, 1].grid(True, alpha=0.3)

    # MRR
    axes[1, 2].plot(epochs, [h["mrr"] for h in history], "-o", color="#e67e22")
    axes[1, 2].set_title("MRR")
    axes[1, 2].set_xlabel("Epoch")
    axes[1, 2].axhline(y=best_mrr, color="r", linestyle="--", alpha=0.5)
    axes[1, 2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(CONFIG["output_dir"] / "training_curves.png", dpi=150)
    plt.show()
else:
    print("No training history to plot.")

## Summary

### v21.3 Training Complete

| Metric | Description |
|--------|-------------|
| Recall@5 | % of synonyms in top-5 |
| Recall@10 | % of synonyms in top-10 |
| MRR | Mean Reciprocal Rank |

### Key Improvements

1. **No saturation**: MRR/Recall metrics show real training progress
2. **Filtered data**: Higher quality synonym pairs
3. **Balanced negatives**: Easy/Medium/Hard difficulty levels

### Output Files

| File | Description |
|------|-------------|
| `best_model.pt` | Best model by MRR |
| `final_model.pt` | Final model |
| `training_history.json` | Training metrics |
| `training_curves.png` | Training plots |

### Next Steps

1. `04_inference_test.ipynb`: Test synonym expansion
2. Compare with v21.2